In [ ]:
##### Copyright 2020 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Visualize the MetaLearning pipeline built on top NitroML. 
# We are using NitroML on Kubeflow: 

This notebook allows users to analyze NitroML metalearning pipelines results.

In [ ]:
# Step 1: Configure your cluster with gcloud
# `gcloud container clusters get-credentials <cluster_name> --zone <cluster-zone> --project <project-id>

# Step 2: Get the port where the gRPC service is running on the cluster
# `kubectl get configmap metadata-grpc-configmap -o jsonpath={.data}`
# Use `METADATA_GRPC_SERVICE_PORT` in the next step. The default port used is 8080.

# Step 3: Port forwarding
# `kubectl port-forward deployment/metadata-grpc-deployment 9898:<METADATA_GRPC_SERVICE_PORT>`

# Troubleshooting
# If getting error related to Metadata (For examples, Transaction already open). Try restarting the metadata-grpc-service using:
# `kubectl rollout restart deployment metadata-grpc-deployment`  

In [ ]:
import sys, os
PROJECT_DIR=os.path.join(sys.path[0], '..')
%cd {PROJECT_DIR}

In [ ]:
import json

import altair as alt
from examples import config as cloud_config
from ml_metadata.proto import metadata_store_pb2
from ml_metadata.metadata_store import metadata_store
from nitroml import results
import tensorflow as tf
import qgrid

## Connect to the ML Metadata (MLMD) database

First we need to connect to our MLMD database which stores the results of our
benchmark runs.

In [ ]:
connection_config = metadata_store_pb2.MetadataStoreClientConfig()

connection_config.host = 'localhost'
connection_config.port = 9898

store = metadata_store.MetadataStore(connection_config)

### Get trial summary data (used to plot Area under Learning Curve) stored as AugmentedTuner artifacts.

In [ ]:
execs = store.get_executions_by_type('nitroml.components.tuner.component.AugmentedTuner')
model_dir_map = {}
for tuner_exec in execs:

    run_id = tuner_exec.properties['run_id'].string_value
    pipeline_root = tuner_exec.properties['pipeline_root'].string_value
    component_id = tuner_exec.properties['component_id'].string_value
    pipeline_name = tuner_exec.properties['pipeline_name'].string_value
    
    if '.run_' not in component_id:
        continue
    if 'dressessales' not in component_id:
        continue
    if 'metalearning_benchmark' != pipeline_name:
        continue
         
    config_path = os.path.join(pipeline_root, component_id, 'trial_summary_plot', str(tuner_exec.id))
    model_dir_map[tuner_exec.id] = config_path

model_dir_map

In [ ]:
d1 = 'gs://artifacts.nitroml-brain-xgcp.appspot.com/examples/examples_metalearning_benchmark/AugmentedTuner.test_OpenML.dressessales.MetaLearningBenchmark.benchmark.OpenML.dressessales.run_1_of_4/trial_summary_plot/4402'
d2 = 'gs://artifacts.nitroml-brain-xgcp.appspot.com/examples/examples_metalearning_benchmark/AugmentedTuner.test_OpenML.dressessales.MetaLearningBenchmark.benchmark.OpenML.dressessales.run_2_of_4/trial_summary_plot/4396'
d3 = 'gs://artifacts.nitroml-brain-xgcp.appspot.com/examples/examples_metalearning_benchmark/AugmentedTuner.test_OpenML.dressessales.MetaLearningBenchmark.benchmark.OpenML.dressessales.run_3_of_4/trial_summary_plot/4401'
d4 = 'gs://artifacts.nitroml-brain-xgcp.appspot.com/examples/examples_metalearning_benchmark/AugmentedTuner.test_OpenML.dressessales.MetaLearningBenchmark.benchmark.OpenML.dressessales.run_4_of_4/trial_summary_plot/4398'

d_list = [d1, d2, d3, d4]

In [ ]:
data_list = []

for d in d_list:
    with tf.io.gfile.GFile(os.path.join(d, 'tuner_plot_data.txt'), mode='r') as fin:
        data_list.append(json.load(fin))

In [ ]:
import examples.tuner_data_utils as tuner_utils
tuner_utils.display_tuner_data_with_error_bars(data_list)

## Analyze the MetaLearner HSpace Output

In [ ]:
execs = store.get_executions_by_type('nitroml.components.metalearning.metalearner.component.MetaLearner')

model_dir_map = {}
for tuner_exec in execs:

    run_id = tuner_exec.properties['run_id'].string_value
    pipeline_root = tuner_exec.properties['pipeline_root'].string_value
    component_id = tuner_exec.properties['component_id'].string_value
    pipeline_name = tuner_exec.properties['pipeline_name'].string_value
    
    # Add filters here
    if '.run_' not in component_id:
        continue
    if 'metalearning_benchmark' != pipeline_name:
        continue
         
    config_path = os.path.join(pipeline_root, component_id, 'output_hyperparameters', str(tuner_exec.id))
    model_dir_map[tuner_exec.id] = config_path

model_dir_map


In [ ]:
# Note the tuner_exec.id or one of the keys from above.
with tf.io.gfile.GFile(os.path.join(model_dir_map[4537], 'meta_hyperparameters.txt'), mode='r') as fin:
    data = json.load(fin)

data['space']

## Display benchmark results

Next we load and visualize `pd.DataFrame` containing our benchmark results.
These results contain contextual features such as the start time, and pipeline
ID, and benchmark metrics as computed by the downstream Evaluators. If your
benchmark included an `EstimatorTrainer` component, its hyperparameters may also
display in the table below.

In [ ]:
#@markdown ### Choose how to aggregate metrics:
mean = False  #@param { type: "boolean" }
stdev = False  #@param { type: "boolean" }
min_and_max = False  #@param { type: "boolean" }

agg = []
if mean:
    agg.append("mean")
if stdev:
    agg.append("std")
if min_and_max:
    agg += ["min", "max"]

df = results.overview(store, metric_aggregators=agg)

In [ ]:
qgid_wdget = qgrid.show_grid(df, show_toolbar=True)
qgid_wdget

In [ ]:
# !jupyter labextension install @jupyterlab/vega5-extension

## Visualize metrics over time

Finally, we plot our benchmark metrics over time. This allows us to monitor how
our model quality changed as we tweaked our pipelines.

If you used the `--runs_per_benchmark` flag, you can even display error bars.

In [ ]:
df